In [ ]:
! pip3 install evaluate
! pip3 install rouge_score
import os
from torch.utils.data import TensorDataset

try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk("..") if "dsait4090_project_location" in files), ".."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

In [ ]:
# !nvidia-smi

In [ ]:
import os
import pandas as pd
from src.common import DATA_PATH, read_data, save_data

LOCAL_DATA_PATH = os.path.join(DATA_PATH, 'custom_decomposition', 'comparison_strategyqa')

train = read_data(os.path.join(LOCAL_DATA_PATH, 'train.json'))
val = read_data(os.path.join(LOCAL_DATA_PATH, 'val.json'))
test = read_data(os.path.join(LOCAL_DATA_PATH, 'test.json'))

# len(train), len(dev)

# p = int(len(train)*0.9)
# test = train[p:]
# train = train[:p]

print(len(train), len(val), len(test))

def process(data):
    new_data = []
    for p in data:
        new_data.append({
            'claim': p['question'],
            'subquestions': '\n'.join(p['decomposition'])
        })

    return new_data

# train = process(train)
# val = process(val)
# test = process(test)

# def save_csv(path, data):
#     df = pd.DataFrame(data)
#     df.to_csv(path, index=True)

# save_csv(os.path.join(LOCAL_DATA_PATH, 'NEW_train.csv'), train)
# save_csv(os.path.join(LOCAL_DATA_PATH, 'NEW_val.csv'), val)
# save_csv(os.path.join(LOCAL_DATA_PATH, 'NEW_test.csv'), test)

In [ ]:
from evaluate import load
#rouge_metric = load('rouge')

In [ ]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
# import pytorch_lightning as pl
import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse
import datetime

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from IPython.display import display, HTML
import random
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset#, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from src.common import DATA_PATH, MODELS_PATH, read_data

nltk.download('punkt')

import wandb
wandb.login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### References: [Official](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/summarization.ipynb#scrollTo=vc0BSBLIIrJQ), [Community](https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb#scrollTo=6R9d7ELIpX9F)

In [ ]:
# BASE_PATH = "/content/drive/MyDrive/DSAIT4090_FinalProject/code/claim-decomp/"
BASE_PATH = LOCAL_DATA_PATH

MODEL_PATH = "google/flan-t5-base"
TOKENIZER_PATH = "google/flan-t5-base"
SAVE_PATH = BASE_PATH + "/test/"
LOGGING_PATH = BASE_PATH + "/test_logs/"
SAVE_MODEL_PATH = BASE_PATH + "/model/"

In [ ]:
import pandas as pd
train = pd.read_csv(BASE_PATH + "/train.csv")
val = pd.read_csv(BASE_PATH + "/val.csv")
test = pd.read_csv(BASE_PATH + "/test.csv")
train

In [ ]:
# data_subq["subquestions"] = data_subq["subquestions"].apply(lambda x: x.replace("[","").replace("]",""))
# data_subq

In [ ]:
# from sklearn.model_selection import train_test_split
# train,val = train_test_split(data_subq,test_size=0.10,random_state=42)
# train

In [ ]:
# train.to_csv("train_strategyq_with_answer.csv",index=False)
# val.to_csv("val_strategyq_with_answer.csv",index=False)

# Loading and processing the data

In [ ]:
TRAIN_FILE_PATH = BASE_PATH + "/train.csv"
VAL_FILE_PATH = BASE_PATH + "/val.csv"
TEST_FILE_PATH = BASE_PATH + "/test.csv"

In [ ]:
data_files = {}
data_files["train"] = TRAIN_FILE_PATH
data_files["validation"] = VAL_FILE_PATH
data_files["test"] = TEST_FILE_PATH

In [ ]:
xmetric = load('rouge')#load_metric("rouge")
data_files
raw_datasets = load_dataset("csv", data_files=data_files)


In [ ]:
metric = load('rouge')#load_metric("rouge")


In [ ]:
raw_datasets["train"][0]

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])


- Classify type of claim

For each type of claim
- Decompose claim
- Get claim prediction

## Tokenization and Dataset Preparation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

In [ ]:
tokenizer

In [ ]:
max_input_length = 64
max_target_length = 128

In [ ]:

def preprocess_function(examples, prefix="decompose the compositional question:"):
    inputs = [prefix + doc for doc in examples["claim"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # The "labels" are the tokenized outputs:
    print(examples["subquestions"])
    labels = tokenizer(text_target=examples["subquestions"], max_length=max_target_length, truncation=True)
    print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

In [ ]:
tokenized_datasets

# Building the model

## Metrics

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions[0])
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print("decoded gen",decoded_preds)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value for key, value in result.items()}#{key: value['f1'] * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Loading the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

In [ ]:
model

## Training Args

In [ ]:
epochs = 36
batch_size = 8
lr = 2e-5

In [ ]:
!pip install accelerate -U

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir=SAVE_PATH,
    learning_rate=lr,
    do_train = True,
    do_eval = True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    num_train_epochs=epochs,
    predict_with_generate=True,
    generation_max_length=512,
    logging_dir=LOGGING_PATH,
    logging_steps=300,
    save_steps=300,
    report_to = "wandb",
)

In [ ]:
wandb_run = wandb.init(
    project="flant5_subq",
    config={
        "per_device_train_batch_size": batch_size,
        "learning_rate": lr})

now = datetime.datetime.now()
current_time = now.strftime("%m/%d/%Y, %H:%M:%S")
wandb_run.name = "run_" + current_time

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

# Training

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
trainer.evaluate() # Evaluation before fine-tuning

In [ ]:
val

In [ ]:
# %%wandb

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
print(SAVE_MODEL_PATH)

In [ ]:
trainer.save_model(SAVE_MODEL_PATH)

In [ ]:
wandb_run.finish()

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_MODEL_PATH)

In [ ]:
model.to('cuda')

In [ ]:
model.device

# Get QuanTemp Subquestions

In [ ]:
def generate_output(test_samples, model):
    inputs = tokenizer(
        test_samples,
        max_length=128,
        return_tensors="pt")

    input_ids = inputs.input_ids.to(model.device)
    #print(input_ids)
    #print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask,min_length = 64, max_length = 128, do_sample=True, top_p=0.95, top_k=50)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(test_samples)
    return output_str

In [ ]:
from src.common import read_data, save_data

train_claims = read_data('raw_data/train_claims.json')
val_claims = read_data('raw_data/val_claims.json')
test_claims = read_data('raw_data/test_claims.json')

In [ ]:
# Test on one sample
print("decompose the compositional question:"+train_claims[0]['claim'], sep="\n", end="\n\n")
output = generate_output("decompose the compositional question:"+train_claims[0]['claim'], model)

In [ ]:
[question + '?' for question in output[0].strip().split('?') if question]

In [ ]:
from tqdm.auto import tqdm

def generate_for_category(claims, category):
  counter = 0
  for claim in tqdm(claims):
    if claim['taxonomy_label'] == category:
      output = generate_output("decompose the compositional question:"+claim['claim'], model)
      subquestions = [question + '?' for question in output[0].strip().split('?') if question]
      claim['subquestions'] = subquestions

  print(counter)
  return claims

# train_claims_subquestions = generate_for_category(train_claims, 'comparison')
val_claims_subquestions = generate_for_category(val_claims, 'comparison')
test_claims_subquestions = generate_for_category(test_claims, 'comparison')


In [ ]:
save_data('custom_decomposition/comparison_strategyqa/train_generated.json', train_claims_subquestions)
save_data('custom_decomposition/comparison_strategyqa/val_generated.json', val_claims_subquestions)
save_data('custom_decomposition/comparison_strategyqa/test_generated.json', test_claims_subquestions)